In [ ]:
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
# To save the frames
output_dir = "/path/to/save/images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Step 1: Define the 3D coordinates of the cube
def create_cube(size=2):
    half_size = size / 2 # Simplify the definition of cube's vertices
    # Define the cube's 8 vertices
    # Each vertex is specified by its (x, y, z) coordinates
    points = np.array([
        [-half_size, -half_size, -half_size], # bottom-left vertex of the front
        [half_size, -half_size, -half_size], # bottom-right vertex of the front
        [half_size, half_size, -half_size], # top-right vertex of the front
        [-half_size, half_size, -half_size], # top-left vertex of the front
        [-half_size, -half_size, half_size], # bottom-left vertex of the back
        [half_size, -half_size, half_size], # bottom-right vertext of the back
        [half_size, half_size, half_size], # top-right vertex of the back
        [-half_size, half_size, half_size], # top-left vertex of the back
    ])
    return points

In [ ]:
# Step 2a: Define the rotation matrix
def rotation_matrix(axis, theta):
    """
    - `axis` is the 3D vector representing the axis of rotation
    - `theta` is the angle of rotation in radians
    """
    # Normalizing the Rotation Axis (so that the rotation is performed around
    # a unit vector)
    axis = axis / np.sqrt(np.dot(axis, axis))
    # Quaternion components (not so sure)
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    # Constructing the rotation matrix
    return np.array([
        [a*a + b*b - c*c - d*d, 2*(b*c - a*d), 2*(b*d + a*c)],
        [2*(b*c + a*d), a*a + c*c - b*b - d*d, 2*(c*d - a*b)],
        [2*(b*d - a*c), 2*(c*d + a*b), a*a + d*d - b*b - c*c]
    ])

In [ ]:
# Step 2b: Define the affine transformation matrix (rotation + translation)
def affine_matrix(rotation_axis, theta, translation):
    """
    - `rotation_axis` is the 3D vector representing the axis of rotation
    - `theta` is the angle of rotation in radians
    - `translation` is a 3D vector representing the translation to be applied
    """
    # Compute the 3x3 rotation matrix
    R = rotation_matrix(rotation_axis, theta)
    # 4x4 identity matrix
    T = np.eye(4)
    # Insert the rotation matrix into the affine transformation matrix
    # (Into the top-left 3x3 submatrix of `T`)
    T[:3, :3] = R
    # "Insert" the translation vector into the affine transformation matrix
    # (Into the fourth column of the top three rows of `T`)
    T[:3, 3] = translation
    return T

In [ ]:
# Step 3: Apply the affine transformation
def apply_affine(points, affine_mat):
    """
    - `points` is an array representing N 3D points
    - `affine_mat` is a 4 x 4 affine transformation matrix
    """
    # Convert the 3D points into homogenous coordinates
    points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1))))
    # Apply the affine transformation
    transformed_points = points_homogeneous @ affine_mat.T # matrix multiplication
    # Convert it back to the 3D coordinates
    return transformed_points[:, :3]

In [ ]:
# Step 4: Define the perspective projection function
def perspective_projection(points):
    projected_points = points.copy()
    # Apply perspective projection to the x-coordinates of the points
    projected_points[:, 0] = points[:, 0] / points[:, 2]
    # Apply perspective projection to the y-coordinates of the points
    projected_points[:, 1] = points[:, 1] / points[:, 2]
    # Return the 2D projected points
    return projected_points[:, :2]

In [ ]:
# 3D Rendering Engine
# Create the cube
cube = create_cube()

# Animation loop
for i, theta in enumerate(np.linspace(0, 2*np.pi, 100)):
    # Set up the affine transformation parameters
    rotation_axis = np.array([0, 1, 0])
    translation = np.array([0, 0, 4])  # Move the cube closer to the camera
    affine_mat = affine_matrix(rotation_axis, theta, translation)

    # Apply the affine transformation
    transformed_cube = apply_affine(cube, affine_mat)

    # Perform perspective projection
    projected_cube = perspective_projection(transformed_cube)

    # Create a blank image
    img_size = (400, 400)
    image = Image.new("RGB", img_size, "white")
    draw = ImageDraw.Draw(image)

    # Center the cube in the image
    center = np.array(img_size) / 2

    # Scale the projected points for better visualization
    scale = 100
    projected_cube = projected_cube * scale + center

    # Draw the cube edges
    edges = [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (0, 4), (1, 5), (2, 6), (3, 7)]
    for edge in edges:
        p1 = tuple(projected_cube[edge[0]])
        p2 = tuple(projected_cube[edge[1]])
        draw.line([p1, p2], fill="black", width=2)

    # Save the current frame as an image
    filename = os.path.join(output_dir, f"frame_{i:03d}.png")
    image.save(filename)